### Flow
- [x] Setup env, import libs
- [ ] Create list of players
- [ ] Download 3 years of OTB games for players
- [ ] Process each game
    - [ ] Cut into FEN
    - [ ] Run Stockfish 15/12 with depth 20/24/30 on FEN, get stats (see list)
    - [ ] Store data
- [ ] Visualize data
    

In [11]:
print('test')

test


In [1]:
# list of players

folder = 'games/'
games = [
    'HansNiemann.pgn',
    'NodirbekYakubboev.pgn',
    'AndreyEsipenko.pgn',
    'SarinNihal.pgn',
    'Praggnanandhaa.pgn',
    'VincentKeymer.pgn',
    'NodirbekAbdusattorov.pgn',
    'ArjunErigaisi.pgn',
    'DommarajuGukesh.pgn',
    'VishyAnand.pgn',
    'FabianoCaruana.pgn',
    'AnishGiri.pgn',
    'HikaruNakamura.pgn',
    'AlirezaFirouzja.pgn',
    'IanNepo.pgn',
    'WesleySo.pgn',
    'DingLiren.pgn',
    'LevonAronian.pgn',
    'MagnusCarlsen.pgn' 
]

In [10]:
stockfish_versions = [
    {
        'release_date': '2022-04-18',# YYYY-MM-DD
        'version': 15,
        'nnue' : True
    },
    {
        'release_date': '2021-07-02',
        'version': 14,
        'nnue' : True
    },
    {
        'release_date': '2021-02-13',
        'version': 13,
        'nnue' : True
    },
    {
        'release_date': '2020-09-02',
        'version': 12,
        'nnue' : True
    },
    {
        'release_date': '2020-01-15', 
        'version': 11,
        'nnue' : False
    },
    {
        'release_date': '2018-12-01',
        'version': 10,
        'nnue' : False
    },
    {
        'release_date': '2018-02-04',
        'version': 9,
        'nnue' : False
    },
]

In [23]:
import chess
import chess.pgn
CATCHFISH_FOLDER="/home/ubuntu/catchfish/"
    
file = CATCHFISH_FOLDER + "games/MagnusCarlsen.pgn"
pgn = open(file)
game = chess.pgn.read_game(pgn)
print(game.headers)
exporter = chess.pgn.StringExporter(headers=False, variations=False, comments=False)
game = game.next()
game = game.next()
game = game.next()
game = game.next()
game = game.next()
game = game.game()
pgn_string = game.accept(exporter)
print(pgn_string)
print(game.end().board().result())

Headers(Event='NOR Championship Group Miniputt', Site='Gausdal NOR', Date='1999.07.03', Round='1', White='Magnus Carlsen', Black='Stefan Randjelovic', Result='0-1', BlackElo='?', ECO='A40', EventDate='1999.07.03', PlyCount='106', WhiteElo='?')
1. d4 e5 2. dxe5 Nc6 3. Bf4 f6 4. exf6 Qxf6 5. Qd2 Qxb2 6. Qe3+ Nge7 7. Be5
Qxc2 8. Bxc7 Nb4 9. Nc3 Na6 10. Rc1 Qb2 11. Qe5 Qxc1+ 12. Nd1 Nxc7 13. e4 Kd8
14. Nf3 Ng6 15. Qg5+ Qxg5 16. Nxg5 Bb4+ 17. Ke2 Nf4+ 18. Kf3 Rf8 19. Nxh7 Nfd5+
20. Nxf8 Bxf8 21. exd5 Nxd5 22. Bc4 Nb6 23. Bb5 a6 24. Bd3 Nd5 25. Re1 Nb4 26.
Bg6 Kc7 27. Re5 Bd6 28. Re4 Nxa2 29. Ne3 Bxh2 30. Rc4+ Kb8 31. Nd5 b5 32. Rc5
Bb7 33. Bf5 Bd6 34. Rc2 Bxd5+ 35. Be4 Bxe4+ 36. Kxe4 Nb4 37. Rd2 Be7 38. Rxd7
Bc5 39. Rxg7 a5 40. Rg5 Be7 41. Rxb5+ Kc8 42. Re5 Ra7 43. Rh5 a4 44. Rh1 a3 45.
Ra1 Nc2 46. Ra2 Ra4+ 47. Kd3 Nb4+ 48. Kc3 Nxa2+ 49. Kb3 Rf4 50. Kxa2 Rxf2+ 51.
Ka1 Rxg2 52. Kb1 a2+ 53. Ka1 Bf6# 0-1
0-1


In [2]:
from stockfish import Stockfish
import chess
import chess.pgn
import time
from dateutil.parser import parse
import datetime
import redis
import sys
import json
from pydash.strings import slugify
from pydash.arrays import reverse
import pprint
prettyPrint = pprint.PrettyPrinter(indent=4).pprint


def parseDate(date):
    try:
        gameDate = datetime.datetime.strptime(date, '%Y.%m.%d')
        return gameDate
    except:
        pass
    try:
        gameDate = datetime.datetime.strptime(date, '%Y-%m-%d')
        return gameDate
    except:
        return False
    
def getTopMoves(fen):  
    tm = None
    
    # check previous evaluation of FEN
    key = "top_moves_" + str(ENGINE_NUM_MOVES) + ":depth_" + str(ENGINE_DEPTH) + ":" + ENGINE_NAME + "_" + str(ENGINE_VERSION) + ":" + fen
    history = redis.get(key)
    if history is not None:
        tm = json.loads(history)
    else:

        # get top five moves
        tic = time.perf_counter()
        tm = stockfish.get_top_moves(ENGINE_NUM_MOVES, include_info=True)
        toc = time.perf_counter()
        print(f"Calculated the top moves in {toc - tic:0.4f} seconds")

        # save to redis
        redis.set(key, json.dumps(tm))
    
    return tm

def getEvaluation(fen):
    # get evaluation (top 1 move?)
    key = "ev:depth_" + str(ENGINE_DEPTH) + ":" + ENGINE_NAME + "_" + str(ENGINE_VERSION) + ":" + fen
    existing = redis.get(key)
    if existing is not None:
        ev = json.loads(existing)
    else:
        etic = time.perf_counter()
        ev = stockfish.get_evaluation()
        etoc = time.perf_counter()
        print(f"Calculated the evalulation in {etoc - etic:0.4f} seconds")

        # save to redis
        redis.set(key, json.dumps(ev))
        
    return ev

def getMoveMade(g):
    try:
        node = g.next()
        if node is not None:
            return node.move
        else:
            return False
    except:
        return False

def getPieceMoved(move_made, sf):
    if move_made is False or None:
        return None
    square = sf.get_what_is_on_square(chess.square_name(move_made.from_square))
    if square is None:
        return None

    return square.name

def getWDLStats(fen):
    key="wdl:depth_" + str(ENGINE_DEPTH) + ":" + ENGINE_NAME + "_" + str(ENGINE_VERSION) + ":" + fen
    existing = redis.get(key)
    if existing is not None:
        wdl = json.loads(existing)
    else:
        etic = time.perf_counter()
        wdl = stockfish.get_wdl_stats()
        etoc = time.perf_counter()
        print(f"Calculated the WDL in {etoc - etic:0.4f} seconds")

        # save to redis
        redis.set(key, json.dumps(wdl))

    return wdl

def getCentipawnLoss(current_move, previous_move):
    
    # check if previous_move was first move, if so, it's 0 cpl
    if previous_move["move_made"] == previous_move["engine"]["top_moves"][0]["Move"]:
        return 0;
    try:    
        # get current eval of position (ie. if playing best move)
        current_evaluation = current_move["engine"]["top_moves"][0]["Centipawn"] # eg. 450 (+4.5)

        # get previous evaluation of position 
        previous_evaluation = previous_move["engine"]["top_moves"][0]["Centipawn"] # eg. 350 (+3.5)

        # previous_move_cpl = previous position evaluation (eg. 350, +3.5) minus current_evaluation (450) = -100 # should never be negative
        # previous_move_cpl = current_evaluation (eg. -134 ) minus previous_evaluation (-154) = 20 change in cp...  

        previous_move_cpl =  abs(current_evaluation - previous_evaluation)
        return previous_move_cpl
    except Exception as e:
        print('getCentipawnLoss error')
        print(e)
        return None

def getStats(moves):
    white_cpls = []
    black_cpls = []
    top_moves_white = {
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0
    }
    top_moves_black = {
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0
    }

    for m in moves:
        if m["white_to_move"] == True:

            # count centipawn losses
            if m["engine"]["centipawnLoss"] is not None:
                white_cpls.append(m["engine"]["centipawnLoss"]) 
        

            # count top moves
            for i, tm in enumerate(m["engine"]["top_moves"], start=1):
                if m["move_made"] == tm["Move"]:
                    top_moves_white[i] += 1
                    
        else:

            # count centipawn losses
            if m["engine"]["centipawnLoss"] is not None:
                black_cpls.append(m["engine"]["centipawnLoss"]) 
            
            # count top moves
            for i, tm in enumerate(m["engine"]["top_moves"], start=1):
                if m["move_made"] == tm["Move"]:
                    top_moves_black[i] += 1
    
 
    acl_white = sum(white_cpls) / len(white_cpls)
    acl_black = sum(black_cpls) / len(black_cpls)
  
    s =  {
        'acl_white' : acl_white,
        'acl_black' : acl_black,
        'top_moves_white' : top_moves_white,
        'top_moves_black' : top_moves_black
    }
    return s


def parseCleanGames(pgn):

    games = []
    while True:

        # read PGN
        try:
            game = chess.pgn.read_game(pgn)
        except:
            continue
        if game is None:
            break  # end of file

        # get date of game
        try:
            gameDate = parseDate(game.headers['Date'])
        except ValueError as ve:
            continue
            
        # only use normal variant
        try:
            # next move
            fen = game.board().fen()
            if fen != chess.STARTING_FEN:
                print('Not a regular chess game, probably 960. Skipping!')
                continue
        except:
            continue

        # use these games
        games.append(game)
    
    return games


def evaluateGames(file):

    pgn = open(file)

    # choose clean games
    games = reverse(parseCleanGames(pgn))

    print("Found", len(games), "games")

    # only evaluate last 100 games
    games = games[-100:]

    count = 0

    # iterate games
    for g in games:

        # debug info
        count += 1
        print("\n--------------------------")
        print('Game', count, '/', len(games))
        for h in g.headers:
            print(h, g.headers[h])
        print('\nDepth:', ENGINE_DEPTH)
        print("\n--------------------------")

        # run evaluation
        evaluateGame(g)

def evaluateGame(g):
    gtic = time.perf_counter()
    
    headers = g.headers
    
    # defaults
    previous_move = None
    previous_best_move = None
    centipawnLoss = None

    moves = []
    while True:
        
        try:
            # next move
            g = g.next() if g.next() is not None else g
        except ValueError as ve:
            continue
            
        if g.next() is None:
            print("No more moves")
            moves.append(previous_move)
            break
        
        # get FEN of position
        fen = g.board().fen()
        
        # set position on board
        stockfish.set_fen_position(fen)
        
        # get top moves
        top_moves = getTopMoves(fen)
        
        # get move made
        move_made = getMoveMade(g)
        
        # # get wdl stats
        # positionWDL = getWDLStats(fen)

        # get evaluation
        try:
            current_best_move_evaluation = top_moves[0]['Centipawn']
        except:
            current_best_move_evaluation = None

        # get piece moved
        piece_moved = getPieceMoved(move_made, stockfish)

        # store move
        current_move = {
            'fen' : fen,
            'fullmove_number' : g.board().fullmove_number,
            'legal_moves_count' : g.board().legal_moves.count(),
            'white_to_move' : g.turn(),
            'move_made' : move_made.uci() if move_made else None,
            'piece_moved' : piece_moved,
            'is_check' : g.board().is_check(),
            'is_capture' : g.board().is_capture(move_made) if move_made else None,
            'is_zeroing_move' : g.board().is_zeroing(move_made) if move_made else None,
            
            'engine' : {
                'name': ENGINE_NAME,
                'version' : ENGINE_VERSION,
                'top_moves' : top_moves,
                'depth' : ENGINE_DEPTH,
                'wdl' : top_moves['WDL'],
                'evaluation' : current_best_move_evaluation,
                'centipawnLoss' : None,
            }
        }

        # in order to store centipawnloss, we need to have evaluated the next position
        # so we're storing 
        if previous_move is not None:
            previous_move['engine']['centipawnLoss'] = getCentipawnLoss(current_move, previous_move)
            moves.append(previous_move)
        else:
            # first move
            moves.append(current_move)

        # prettyPrint(previous_move)

        # store
        previous_move = current_move

    # cleanup headers
    cleanHeaders = {}
    for h in headers:
        cleanHeaders[h] = headers[h]

    # calc stats
    try:
        stats = getStats(moves)
    except Exception as e:
        stats = None
        print('stats error', e)
        pass

    exporter = chess.pgn.StringExporter(headers=False, variations=False, comments=False)
    pgn_string = g.game().accept(exporter)

    # stored object
    evaluatedGame = {
        'moves' : moves,
        'engine' : {
            'parameters' : stockfish.get_parameters(),
            'name': ENGINE_NAME,
            'version' : ENGINE_VERSION,
        },
        'stats': stats,
        'headers' : cleanHeaders,
        'pgn' : pgn_string,
        'result' : g.game().end().board().result(),
    }
   
    print("Evaluated", len(moves), "moves")
    print("Stats", stats)

    gtoc = time.perf_counter()
    print('\n')
    print(f"## Calculated the game to in {gtoc - gtic:0.4f} seconds")
    print('\n')
    
    filevars = cleanHeaders['Event'] + '-' + cleanHeaders['Site'] + '-' + cleanHeaders['Date'] + '-' + cleanHeaders['White'] + '-vs-' + cleanHeaders['Black'] + '-ply' + cleanHeaders['PlyCount'] + '-round' + cleanHeaders['Round'] + '.depth' + str(ENGINE_DEPTH) + '.' + ENGINE_NAME + str(ENGINE_VERSION) 
    gamefile = CATCHFISH_FOLDER +  "games/evals/"  + slugify(filevars) + '.evaluation'
    with open(gamefile, 'w') as gf:
        gf.write(json.dumps(evaluatedGame))



# globals
ENGINE_DEPTH=20
ENGINE_NUM_MOVES=5
ENGINE_NAME="Stockfish"
ENGINE_CPU_THREADS=196
ENGINE_HASH_SIZE=2048
CATCHFISH_FOLDER="/home/ubuntu/catchfish/"
    
# init stockfish
stockfish = Stockfish(

    # path to stockfish binary
    path="/home/ubuntu/catchfish/stockfish_15_linux_x64_avx2/stockfish_15_x64_avx2", 
    
    # calculate all moves to this depth
    depth=ENGINE_DEPTH, 
    
    # stockfish settings
    parameters={
      "Threads": ENGINE_CPU_THREADS, # cpu threads
      "Minimum Thinking Time": 1,
      "Debug Log File" : "/home/ubuntu/catchfish/debug.log",
      "Hash" : ENGINE_HASH_SIZE, 
      "Ponder" : False,
      "MultiPV" : ENGINE_NUM_MOVES, # lines
  }
)

stockfish.set_skill_level(20)

# print settings
print("Stockfish settings:")
prettyPrint(stockfish.get_parameters())

ENGINE_VERSION = stockfish.get_stockfish_major_version()

# init redis
redis = redis.Redis(
    host= 'localhost',
    port= '6379',
    db = 3
)


players = [
    'HansNiemann.pgn',
    'NodirbekYakubboev.pgn',
    'AndreyEsipenko.pgn',
    'SarinNihal.pgn',
    'Praggnanandhaa.pgn',
    'VincentKeymer.pgn',
    'NodirbekAbdusattorov.pgn',
    'ArjunErigaisi.pgn',
    'DommarajuGukesh.pgn',
    'VishyAnand.pgn',
    'FabianoCaruana.pgn',
    'AnishGiri.pgn',
    'HikaruNakamura.pgn',
    'AlirezaFirouzja.pgn',
    'IanNepo.pgn',
    'WesleySo.pgn',
    'DingLiren.pgn',
    'LevonAronian.pgn',
    'MagnusCarlsen.pgn'
]

for d in [15, 20, 24]:
    ENGINE_DEPTH = d
    print("Evaluating depth", ENGINE_DEPTH)
    stockfish.set_depth(ENGINE_DEPTH)

    for player in players:
        file = CATCHFISH_FOLDER + "games/" + player
        print("Processing", file)
        evaluateGames(file)

print("\nDone!")


# improvements
# - fix 1000 evaluations, dont go so deep, billions of nodes
# - remove 'evaluation', info already in top engine move
# - use historical stockfish versions and nps of iphone 7, 11, laptop, etc. find sweet spot in analysis.
# - use cluster

Stockfish settings:
{   'Contempt': 0,
    'Debug Log File': '/home/ubuntu/catchfish/debug.log',
    'Hash': 2048,
    'Min Split Depth': 0,
    'Minimum Thinking Time': 1,
    'Move Overhead': 10,
    'MultiPV': 5,
    'Ponder': False,
    'Skill Level': 20,
    'Slow Mover': 100,
    'Threads': 196,
    'UCI_Chess960': 'false',
    'UCI_Elo': 1350,
    'UCI_LimitStrength': 'false'}
Evaluating depth 15
Processing /home/ubuntu/catchfish/games/HansNiemann.pgn
Found 615 games

--------------------------
Game 1 / 100
Event St Louis Inv GM 2018
Site Saint Louis USA
Date 2018.11.19
Round 7.1
White Hans Moke Niemann
Black Ali Marandi, Cemil Can
Result 1-0
BlackElo 2552
ECO A84
EventDate ?
PlyCount 120
WhiteElo 2439

Depth: 15

--------------------------
Calculated the top moves in 0.2701 seconds
Calculated the WDL in 0.1769 seconds
Calculated the top moves in 0.3615 seconds
Calculated the WDL in 0.0828 seconds
Calculated the top moves in 0.3941 seconds
Calculated the WDL in 0.1273 seconds
Ca

: 